In [1]:
import sys,os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))) # 添加上层目录到 sys.path
# current_directory = os.path.abspath(os.path.curdir)
# parent_directory = os.path.dirname(current_directory)
# sys.path.append(parent_directory)
from data_process import X_test,X_train,X_valid,y_test_master
from data_process import y_test,y_train,y_train_master,y_valid_master,y_valid
from data_process import X_valid_MCI,X_train_MCI,y_valid_MCI,y_train_MCI
# from dimension_reduction import X_train,X_valid,X_test
from dimension_reduction_lda import dimension_reduction,visualization
from train_and_test import train_and_test,device
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn

{'AD': 0, 'MCI': 1, 'MCIn': 2, 'MCIp': 3, 'NC': 4}
random seed: 1697033146


In [2]:
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp

def construct_knn_graph(X,y, k=5, metric='unsupervised'):
    '''
    Description: Construct the knn graph of the data.
    
    Input:
    - X: Train data and test data.
    - y: The label of train data.
    - k: The number of nearest neighbors.
    
    Return:
        - the knn graph of the data.
    '''
    knn=NearestNeighbors(n_neighbors=k,metric=metric)
    knn.fit(X)
    _, indices = knn.kneighbors(X) # Indices of the nearest points in the population matrix.
    adj_mat=np.zeros((len(X),len(X)))
    for i in range(len(X)):
        for j in range(k):
            adj_mat[i][indices[i][j]]=1
    edge_index = []
    for i in range(len(X_train)):
        for j in range(len(X_train)):
            if j>i and adj_mat[i][j]==1:
                edge_index.append([i,j])
    edge_index = torch.tensor(edge_index).T
    edge_index = edge_index.to(device)
    X = torch.tensor(X_train).float().to(device)
    y = torch.tensor(y_train).long().to(device)
    data = pyg_nn.data.Data(x=X,edge_index=edge_index,y=y)
    return data

In [4]:
import argparse
import torch_geometric.transforms as T
parser = argparse.ArgumentParser()
parser.add_argument('--use_gdc', action='store_true',
                    help='Use GDC preprocessing.')
args = parser.parse_args()
if args.use_gdc:
    gdc = T.GDC(self_loop_weight=1, normalization_in='sym',
                normalization_out='col',
                diffusion_kwargs=dict(method='ppr', alpha=0.05),
                sparsification_kwargs=dict(method='topk', k=128,
                                           dim=0), exact=True)
    data = gdc(data)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # GCNConv模型输入参数：输入结点特征维度，输出结点特征维度，是否cached和是否normalize；
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True,
                             normalize=not args.use_gdc)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True,
                             normalize=not args.use_gdc)

        self.reg_params = self.conv1.parameters()
        self.non_reg_params = self.conv2.parameters()

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)  # dropout操作，避免过拟合
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)  # 模型最后一层接上一个softmax和CNN类似

usage: ipykernel_launcher.py [-h] [--use_gdc]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"3cc5f0db-f2a9-4007-a7b7-5622797c9b51" --shell=9017 --transport="tcp" --iopub=9019 --f=c:\Users\Calculus\AppData\Roaming\jupyter\runtime\kernel-v2-17236X5bIaLSWNx2J.json


SystemExit: 2